In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_json('script-bag-of-words.json')

In [ ]:
df.tail()

,episodeAlt,seasonNum,episodeNum,episodeTitle,text
68,S8E2,8,2,A Knight of the Seven Kingdoms,"[{'name': 'Daenerys Targaryen', 'text': 'About..."
69,S8E3,8,3,The Long Night,"[{'name': 'Northman #1', 'text': 'Oi!'}, {'nam..."
70,S8E4,8,4,The Last of the Starks,"[{'name': 'Jon Snow', 'text': 'And Everyone It..."
71,S8E5,8,5,The Bells,"[{'name': 'Lord Varys', 'text': 'And? Come Not..."
72,S8E6,8,6,The Iron Throne,"[{'name': 'Tyrion Lannister', 'text': 'I'll fi..."


In [ ]:
df.iloc[-1]['text']

[{'name': 'Tyrion Lannister', 'text': "I'll find later. you"},
 {'name': 'Jon Snow', 'text': "It's Let me men not safe. send some with you."},
 {'name': 'Tyrion Lannister', 'text': "I'm alone. going"},
 {'name': 'Grey Worm',
  'text': 'Daenerys I In Queen, Targaryen, die. name of one sentence the the to true you'},
 {'name': 'Jon Snow',
  'text': "Grey It's These Worm! are men over. prisoners."},
 {'name': 'Grey Worm',
  'text': "It Queen's are defeated. enemies is not over the until"},
 {'name': 'Davos Seaworth',
  'text': "How They're be? defeated do knees. more much on their them to want you"},
 {'name': 'Grey Worm', 'text': 'They are breathing.'},
 {'name': 'Davos Seaworth', 'text': 'Look We around friend. won. you,'},
 {'name': 'Grey Worm', 'text': "I commands, my not obey queen's yours."},
 {'name': 'Jon Snow', 'text': "And Queen's are commands? the what"},
 {'name': 'Grey Worm',
  'text': '"Kill Cersei Lannister." These They all are chose fight follow for free her. men. to who'}

In [ ]:
dialogue = {}
for index, row in df.iterrows():
  for item in row['text']:
    if item['name'] in dialogue:
    #append
      dialogue[item['name']] = dialogue[item['name']]+item['text']
    else:
      dialogue[item['name']] = item['text']+" "

In [ ]:
len(dialogue)

817

In [ ]:
new_df = pd.DataFrame()
new_df['character'] = dialogue.keys()
new_df['words'] = dialogue.values()

In [ ]:
new_df.iloc[:,0:3].head(5)

,character,words,num_words
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823


In [ ]:
new_df['num_words'] = new_df['words'].apply(lambda x:len(x.split()))

In [ ]:
new_df =new_df.sort_values('num_words', ascending = False)

In [ ]:
new_df= new_df.head(100)

In [ ]:
new_df.shape

(100, 3)

In [ ]:
new_df.head()

,character,words,num_words
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [ ]:
embeddings = cv.fit_transform(new_df['words']).toarray()

In [ ]:
embeddings.shape

(100, 15335)

In [ ]:
embeddings = embeddings.astype('float64')

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(embeddings)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 100 samples in 0.002s...
[t-SNE] Computed neighbors for 100 samples in 0.191s...
[t-SNE] Computed conditional probabilities for sample 100 / 100
[t-SNE] Mean sigma: 14.377887
[t-SNE] KL divergence after 250 iterations with early exaggeration: 64.401100
[t-SNE] KL divergence after 800 iterations: 0.301372


In [ ]:
z.shape

(100, 2)

In [ ]:
new_df['x'] = z.T[0]
new_df['y'] = z.T[1]


In [ ]:
new_df

,character,words,num_words,x,y
17,Tyrion Lannister,It Mmh. Northern about girls. is say the they ...,25924,-3.342438,0.753784
13,Cersei Lannister,And And Casterly One Rock. When about afraid. ...,14294,-3.092457,0.733753
3,Jon Snow,Father's Go on. watching. And mother. yourBran...,11488,-2.974039,0.970400
20,Daenerys Targaryen,We've a and anything. asked been for for guest...,11202,-2.586169,0.337781
12,Jaime Lannister,"As I It's brother, duty feel it's much. my sho...",10823,-2.908238,0.824755
...,...,...,...,...,...
132,Rickard Karstark,Castle I'll King North! Red The They and can c...,466,2.763872,3.781367
57,Syrio Forel,"Tomorrow You are at be boy. here late, midday....",462,3.403694,2.542524
112,Kevan Lannister,Tyrion. Catelyn Golden Jaime Lords River River...,460,3.141036,2.772262
365,Mace Tyrell,"From Grace, House Margaery May Reach, Tyrell Y...",450,3.128660,2.601798


In [ ]:
import plotly.express as px
fig = px.scatter(new_df.head(25), x="x", y="y", color="character")
fig.show()